분석에 적합하게 전처리하는 단계로 순서는 다음과 같다.<br>
fail된 disk_id의 결측치를 채운다.<br>
시간에 안맞는 disk_id를 제거한다.<br>
fail이후의 disk_id는 지운다.<br>
fail전 30일을 1로 라벨링한다.<br>
그리고 failed된 disk_id개수만큼 healthy한 디스크 아이디를 가져와서 concat한다.<br>

In [98]:
import pandas as pd
import numpy as np

data = pd.read_csv('../ssd_data/combined_by_diskmodel/combined_MB1.csv',index_col=False)

# 데이터를 시간 순서대로 정렬 ('ds' 컬럼을 기준으로)
data = data.sort_values(by=['ds'])
data = data.drop_duplicates(subset=['disk_id', 'ds'])
data.reset_index(inplace=True)
del(data['index'])
del(data['level_0'])
data

,disk_id,ds,model,n_1,r_1,n_5,r_5,n_9,r_9,n_12,...,r_244,n_245,r_245,n_175,r_175,n_232,r_232,label,n_211,r_211
0,10005,20190101,MB1,NaN,NaN,100.0,0.0,99.0,4600.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
1,39138,20190101,MB1,NaN,NaN,100.0,0.0,97.0,13256.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
2,3918,20190101,MB1,NaN,NaN,100.0,0.0,97.0,13305.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
3,39187,20190101,MB1,NaN,NaN,100.0,0.0,97.0,12934.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
4,39189,20190101,MB1,NaN,NaN,100.0,0.0,97.0,12934.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7132648,27627,20190630,MB1,NaN,NaN,100.0,0.0,96.0,17027.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
7132649,27631,20190630,MB1,NaN,NaN,100.0,0.0,96.0,17027.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
7132650,27676,20190630,MB1,NaN,NaN,100.0,0.0,96.0,18069.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN
7132651,27586,20190630,MB1,NaN,NaN,100.0,0.0,98.0,8476.0,99.0,...,0.0,100.0,65535.0,NaN,NaN,NaN,NaN,0,NaN,NaN


In [99]:
# null값이 있는 열을 제거하는 코드
def remove_null_columns(df):
    # null인 컬럼을 확인하고, 모든 값이 null인 컬럼을 찾습니다.
    null_columns = df.columns[df.isnull().all()]
    
    # 모든 값이 null인 컬럼을 DataFrame에서 제거합니다.
    df = df.drop(null_columns, axis=1)
    
    return df
# 하나의 unique한 값으로만 이루어진 컬럼을 제거하는 코드
def remove_single_value_columns(df):
    single_value_cols = []
    for col in df.columns:
        unique_values = df[col].nunique()
        if unique_values == 1:
            single_value_cols.append(col)

    df_filtered = df.drop(single_value_cols, axis=1)
    return df_filtered


In [100]:
data = remove_null_columns(data)
data = remove_single_value_columns(data)

failed된 disk_id중에서 첫 관측에서 마지막 관측 사이의 결측치 채우기<br>
interpolate-linear방식으로

In [71]:
selected_disk_ids = data[data['label']==1]['disk_id'].unique()

dfs_to_concat = []  # 추가할 데이터프레임들을 저장할 리스트

for disk_id in selected_disk_ids:
    disk_data = data[data['disk_id'] == disk_id].copy()
    disk_data['ds'] = pd.to_datetime(disk_data['ds'], format='%Y%m%d')
    disk_data = disk_data.sort_values(by='ds')
    disk_data = disk_data.set_index('ds', drop=True)
    all_dates = pd.date_range(start=disk_data.index.min(), end=disk_data.index.max(), freq='D')
    disk_data = disk_data.reindex(all_dates)
    disk_data = disk_data.interpolate(method='linear')
    dfs_to_concat.append(disk_data)

interpolated_failed_data = pd.concat(dfs_to_concat)  # 데이터프레임들을 연결하여 새로운 데이터프레임 생성

interpolated_failed_data = interpolated_failed_data.reset_index()  # 인덱스 재설정
interpolated_failed_data

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
0,2019-01-01,17909.0,99.0,2.0,97.0,12819.0,99.0,19.0,97.0,155.0,...,32.0,199.0,1.0,100.0,0.0,99.0,4.761304e+11,99.0,3.422788e+11,1.0
1,2019-01-02,17909.0,99.0,2.0,97.0,12843.0,99.0,19.0,97.0,155.0,...,31.0,199.0,1.0,100.0,0.0,99.0,4.774871e+11,99.0,3.432220e+11,0.0
2,2019-01-03,17909.0,99.0,2.0,97.0,12867.0,99.0,19.0,97.0,156.0,...,31.0,199.0,1.0,100.0,0.0,99.0,4.790559e+11,99.0,3.467700e+11,0.0
3,2019-01-04,17909.0,99.0,2.0,97.0,12891.0,99.0,19.0,97.0,156.0,...,32.0,199.0,1.0,100.0,0.0,99.0,4.805626e+11,99.0,3.477830e+11,0.0
4,2019-01-05,17909.0,99.0,2.0,97.0,12915.0,99.0,19.0,97.0,157.0,...,31.0,199.0,1.0,100.0,0.0,99.0,4.819720e+11,99.0,3.512526e+11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43895,2019-06-26,8599.0,100.0,0.0,96.0,17168.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201000e+12,99.0,8.356453e+11,0.0
43896,2019-06-27,8599.0,100.0,0.0,96.0,17192.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201312e+12,99.0,8.362658e+11,0.0
43897,2019-06-28,8599.0,100.0,0.0,96.0,17216.0,99.0,21.0,91.0,624.0,...,31.0,200.0,0.0,100.0,0.0,99.0,2.201585e+12,99.0,8.371074e+11,0.0
43898,2019-06-29,8599.0,99.0,2.0,96.0,17240.0,99.0,21.0,91.0,624.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201871e+12,99.0,8.379656e+11,0.0


failed된 시점부터 30일전이 없으면 disk_id를 제거한다

In [72]:
selected_disk_model_1 = interpolated_failed_data['disk_id'].unique()

to_delete_id =[]
for disk_id in selected_disk_model_1:
    disk_data = interpolated_failed_data[interpolated_failed_data['disk_id'] == disk_id].copy()
    disk_data = disk_data.sort_values(by='index')
    disk_data.reset_index(inplace=True,drop=True)

    failed_index = disk_data[disk_data['label']==1]['index'].index.min()
    start_index = disk_data.index.min()

    if(failed_index - start_index) < 30:
        to_delete_id.append(disk_id)

# failed_disk_index에 해당하는 행들을 제거
interpolated_failed_data_over30 = interpolated_failed_data[~interpolated_failed_data['disk_id'].isin(to_delete_id)]
interpolated_failed_data_over30

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
4148,2019-01-01,41400.0,100.0,0.0,97.0,12694.0,99.0,13.0,99.0,55.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.812450e+11,99.0,1.329623e+11,0.0
4149,2019-01-02,41400.0,100.0,0.0,97.0,12718.0,99.0,13.0,99.0,55.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.813687e+11,99.0,1.333035e+11,0.0
4150,2019-01-03,41400.0,100.0,0.0,97.0,12742.0,99.0,13.0,99.0,56.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.815201e+11,99.0,1.336394e+11,0.0
4151,2019-01-04,41400.0,100.0,0.0,97.0,12766.0,99.0,13.0,99.0,56.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.816743e+11,99.0,1.339799e+11,0.0
4152,2019-01-05,41400.0,100.0,0.0,97.0,12790.0,99.0,13.0,99.0,56.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.818216e+11,99.0,1.343434e+11,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43895,2019-06-26,8599.0,100.0,0.0,96.0,17168.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201000e+12,99.0,8.356453e+11,0.0
43896,2019-06-27,8599.0,100.0,0.0,96.0,17192.0,99.0,21.0,91.0,623.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201312e+12,99.0,8.362658e+11,0.0
43897,2019-06-28,8599.0,100.0,0.0,96.0,17216.0,99.0,21.0,91.0,624.0,...,31.0,200.0,0.0,100.0,0.0,99.0,2.201585e+12,99.0,8.371074e+11,0.0
43898,2019-06-29,8599.0,99.0,2.0,96.0,17240.0,99.0,21.0,91.0,624.0,...,30.0,200.0,0.0,100.0,0.0,99.0,2.201871e+12,99.0,8.379656e+11,0.0


failed된 disk중 fail이후에 있는 디스크는 제거한다.

In [83]:
# index 컬럼에 따라 정렬
interpolated_failed_data_over30 = interpolated_failed_data_over30.sort_values(by=['disk_id','index'])
# 결과 출력
interpolated_failed_data_over30

interpolated_failed_data_over30['index'] = pd.to_datetime(interpolated_failed_data_over30['index'], format='%Y%m%d')

remove_indexes = []
for index, row in interpolated_failed_data_over30.iterrows():
    if row['label'] == 1:
        disk_id = row['disk_id']
        index = row['index']
        remove_indexes.extend(interpolated_failed_data_over30.index[(interpolated_failed_data_over30['disk_id'] == disk_id) & (interpolated_failed_data_over30['index'] > index)].tolist())

# 중복된 인덱스를 제거하여 제거할 행들을 구함
remove_indexes = list(set(remove_indexes))

# 제거할 행을 제외한 최종 DataFrame을 생성
del_fail_after_row = interpolated_failed_data_over30.drop(index=remove_indexes)

# 결과 출력
del_fail_after_row

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
12656,2019-01-01,470.0,99.0,4.0,97.0,14280.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.543009e+09,0.0
12657,2019-01-02,470.0,99.0,4.0,97.0,14304.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.544827e+09,0.0
12658,2019-01-03,470.0,99.0,4.0,97.0,14328.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.546758e+09,0.0
12659,2019-01-04,470.0,99.0,4.0,97.0,14352.0,99.0,18.0,99.0,7.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.548679e+09,0.0
12660,2019-01-05,470.0,99.0,4.0,97.0,14376.0,99.0,18.0,99.0,7.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.550465e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,2019-03-21,44338.0,99.0,2.0,96.0,16176.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.778025e+09,0.0
15345,2019-03-22,44338.0,99.0,2.0,96.0,16200.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.779657e+09,0.0
15346,2019-03-23,44338.0,99.0,2.0,96.0,16224.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.781085e+09,0.0
15347,2019-03-24,44338.0,99.0,2.0,96.0,16248.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.782419e+09,0.0


fail이 일어난 시점부터 30일전을 1로 라벨링하는 작업

In [84]:
# 'index' 컬럼을 datetime 형태로 변환
del_fail_after_row['index'] = pd.to_datetime(del_fail_after_row['index'], format='%Y%m%d')

# 모든 행에 대해 반복
for index, row in del_fail_after_row.iterrows():
    # label 값이 1인 경우
    if row['label'] == 1:
        # 해당 행의 disk_id
        current_disk_id = row['disk_id']
        # 해당 행의 날짜
        current_date = row['index']
        # 30일 이전의 날짜 계산
        days_before = current_date - pd.Timedelta(days=30)
        
        # 같은 disk_id이면서, 날짜가 30일 이내인 행들을 찾아서 label 값을 1로 설정
        del_fail_after_row.loc[(del_fail_after_row['disk_id'] == current_disk_id) &
                             (del_fail_after_row['index'] <= current_date) &
                             (del_fail_after_row['index'] > days_before), 'label'] = 1

In [87]:
del_fail_after_row

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
12656,2019-01-01,470.0,99.0,4.0,97.0,14280.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.543009e+09,0.0
12657,2019-01-02,470.0,99.0,4.0,97.0,14304.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.544827e+09,0.0
12658,2019-01-03,470.0,99.0,4.0,97.0,14328.0,99.0,18.0,99.0,7.0,...,28.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.546758e+09,0.0
12659,2019-01-04,470.0,99.0,4.0,97.0,14352.0,99.0,18.0,99.0,7.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.548679e+09,0.0
12660,2019-01-05,470.0,99.0,4.0,97.0,14376.0,99.0,18.0,99.0,7.0,...,29.0,200.0,0.0,100.0,0.0,99.0,1.383233e+10,99.0,8.550465e+09,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,2019-03-21,44338.0,99.0,2.0,96.0,16176.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.778025e+09,1.0
15345,2019-03-22,44338.0,99.0,2.0,96.0,16200.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.779657e+09,1.0
15346,2019-03-23,44338.0,99.0,2.0,96.0,16224.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.781085e+09,1.0
15347,2019-03-24,44338.0,99.0,2.0,96.0,16248.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.782419e+09,1.0


fail한 디스크아이디는 242개라서 healthy한 디스크아이디도 242개를 가져오겠다.<br>
180개로 이루어진 disk_id를 가져오고 보간까지해서 181개로 만들겠다

In [107]:
import random
random.seed(42)  # 랜덤 선택 결과 재현을 위한 시드 설정

value_count = data[data['label'] == 0]['disk_id'].value_counts()
healthy_disk_id_180 = value_count[value_count == 180].index.tolist()
selected_ids = random.sample(healthy_disk_id_180, 242)  # unique_disk_ids에서 랜덤으로 304개 선택

selected_disk_ids = pd.Series(selected_ids, name='disk_id')
selected_disk_ids

0       6653
1      23765
2        445
3      10603
4      32326
       ...  
237    24683
238    12097
239    41811
240    10724
241    41875
Name: disk_id, Length: 242, dtype: int64

In [108]:
dfs_to_concat = []  # 추가할 데이터프레임들을 저장할 리스트

for disk_id in selected_disk_ids:
    disk_data = data[data['disk_id'] == disk_id].copy()
    disk_data['ds'] = pd.to_datetime(disk_data['ds'], format='%Y%m%d')
    disk_data = disk_data.sort_values(by='ds')
    disk_data = disk_data.set_index('ds', drop=True)
    all_dates = pd.date_range(start=disk_data.index.min(), end=disk_data.index.max(), freq='D')
    disk_data = disk_data.reindex(all_dates)
    disk_data = disk_data.interpolate(method='linear')
    dfs_to_concat.append(disk_data)

selected_filtered_data_healthy = pd.concat(dfs_to_concat)  # 데이터프레임들을 연결하여 새로운 데이터프레임 생성

selected_filtered_data_healthy = selected_filtered_data_healthy.reset_index()  # 인덱스 재설정

In [110]:
selected_filtered_data_healthy

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
0,2019-01-01,6653.0,100.0,0.0,97.0,12707.0,99.0,12.0,99.0,28.0,...,28.0,200.0,0.0,100.0,0.0,99.0,4.976439e+10,99.0,3.939800e+10,0.0
1,2019-01-02,6653.0,100.0,0.0,97.0,12731.0,99.0,12.0,99.0,28.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.034821e+10,99.0,3.956458e+10,0.0
2,2019-01-03,6653.0,100.0,0.0,97.0,12755.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.082254e+10,99.0,3.973763e+10,0.0
3,2019-01-04,6653.0,100.0,0.0,97.0,12779.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.121309e+10,99.0,3.990997e+10,0.0
4,2019-01-05,6653.0,100.0,0.0,97.0,12803.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.157860e+10,99.0,4.008257e+10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43797,2019-06-26,41875.0,100.0,0.0,98.0,7555.0,99.0,23.0,98.0,82.0,...,35.0,200.0,0.0,100.0,0.0,99.0,1.885601e+11,99.0,7.028654e+10,0.0
43798,2019-06-27,41875.0,100.0,0.0,98.0,7579.0,99.0,23.0,98.0,82.0,...,35.0,200.0,0.0,100.0,0.0,99.0,1.890818e+11,99.0,7.082021e+10,0.0
43799,2019-06-28,41875.0,100.0,0.0,98.0,7603.0,99.0,23.0,98.0,83.0,...,35.0,200.0,0.0,100.0,0.0,99.0,1.896921e+11,99.0,7.134874e+10,0.0
43800,2019-06-29,41875.0,100.0,0.0,98.0,7627.0,99.0,23.0,98.0,83.0,...,32.0,200.0,0.0,100.0,0.0,99.0,1.902951e+11,99.0,7.171210e+10,0.0


concat을 하겠다

In [113]:
result = pd.concat([selected_filtered_data_healthy, del_fail_after_row])
result

,index,disk_id,n_5,r_5,n_9,r_9,n_12,r_12,n_177,r_177,...,r_194,n_195,r_195,n_199,r_199,n_241,r_241,n_242,r_242,label
0,2019-01-01,6653.0,100.0,0.0,97.0,12707.0,99.0,12.0,99.0,28.0,...,28.0,200.0,0.0,100.0,0.0,99.0,4.976439e+10,99.0,3.939800e+10,0.0
1,2019-01-02,6653.0,100.0,0.0,97.0,12731.0,99.0,12.0,99.0,28.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.034821e+10,99.0,3.956458e+10,0.0
2,2019-01-03,6653.0,100.0,0.0,97.0,12755.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.082254e+10,99.0,3.973763e+10,0.0
3,2019-01-04,6653.0,100.0,0.0,97.0,12779.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.121309e+10,99.0,3.990997e+10,0.0
4,2019-01-05,6653.0,100.0,0.0,97.0,12803.0,99.0,12.0,99.0,29.0,...,28.0,200.0,0.0,100.0,0.0,99.0,5.157860e+10,99.0,4.008257e+10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,2019-03-21,44338.0,99.0,2.0,96.0,16176.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.778025e+09,1.0
15345,2019-03-22,44338.0,99.0,2.0,96.0,16200.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.779657e+09,1.0
15346,2019-03-23,44338.0,99.0,2.0,96.0,16224.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.781085e+09,1.0
15347,2019-03-24,44338.0,99.0,2.0,96.0,16248.0,99.0,21.0,99.0,6.0,...,31.0,200.0,0.0,100.0,0.0,99.0,1.352326e+10,99.0,8.782419e+09,1.0


In [114]:
result.to_csv("../ssd_data/final_data/final_MB1.csv",index = False)